In [ ]:
import os
import subprocess
import time
import ssl
import httplib2
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, HttpRequest
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials

# ✅ Disable SSL globally
ssl._create_default_https_context = ssl._create_unverified_context

# ✅ Force httplib2 to ignore SSL certificate verification
httplib2.Http(disable_ssl_certificate_validation=True)

# Google API configuration
SCOPES = ["https://www.googleapis.com/auth/youtube.upload"]
CLIENT_SECRETS_FILE = "client_secret.json"
TOKEN_FILE = "token.json"
YOUTUBE_MAX_DURATION = 43200  # 12 hours in seconds

MERGE_TIMEOUT = 1200  # 20 minutes
CONVERT_TIMEOUT = 7200  # 2 hours

def get_authenticated_service():
    """
    Authenticate with YouTube API and return a service object.
    If re-authentication is required, open a browser before starting the process.
    """
    creds = None

    # ✅ Check if a token file exists
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE)

    # ✅ If token is missing or invalid, refresh or re-authenticate before merging starts
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            try:
                print("🔄 Token expired. Attempting to refresh...")
                creds.refresh(Request())
                print("✅ Token refreshed successfully!")
            except Exception as e:
                print(f"⚠️ Token refresh failed: {e}\n🌐 Opening browser for re-authentication...")
                creds = None  # Force re-authentication

        if creds is None:
            print("🌐 Opening browser for authentication...")
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
            creds = flow.run_local_server(port=0)

        # ✅ Save the new token immediately
        with open(TOKEN_FILE, "w") as token_file:
            token_file.write(creds.to_json())

    # ✅ Force httplib2 (used by googleapiclient) to ignore SSL verification
    return build("youtube", "v3", credentials=creds, cache_discovery=False, requestBuilder=lambda *args, **kwargs: HttpRequest(*args, http=httplib2.Http(disable_ssl_certificate_validation=True), **kwargs))

# 🔥 Call authentication before any conversion or merging starts!
youtube_service = get_authenticated_service()
print("✅ Authentication successful! Now processing videos...")

def upload_to_youtube(video_path, title, description, folder_path):
    """
    Uploads a video to YouTube and saves the video URL in 'youtube link.txt'.
    """
    print(f"📤 Uploading: {title}...")
    youtube = get_authenticated_service()
    request = youtube.videos().insert(
        part="snippet,status",
        body={"snippet": {"title": title, "description": description, "categoryId": "27"}, "status": {"privacyStatus": "unlisted"}},
        media_body=MediaFileUpload(video_path, chunksize=-1, resumable=True)
    )

    _, response = request.next_chunk()
    video_url = f"https://www.youtube.com/watch?v={response['id']}"
    with open(os.path.join(folder_path, "youtube_link.txt"), "w") as f:
        f.write(video_url + "\n")

    print(f"✅ Upload complete! Video URL: {video_url}")
    return video_url

if __name__ == "__main__":
    parent_folder = input("📂 Enter the parent folder containing all video subfolders: ").strip()

    for subfolder in sorted(os.listdir(parent_folder)):
        subfolder_path = os.path.join(parent_folder, subfolder)

        if os.path.isdir(subfolder_path):
            print(f"🚀 Processing folder: {subfolder_path}")

            video_files = sorted([os.path.join(subfolder_path, f) for f in os.listdir(subfolder_path) if f.endswith('.mp4')])
            merged_video_path, description, _ = merge_videos_and_create_timestamps(subfolder_path, video_files)
            if merged_video_path:
                upload_to_youtube(merged_video_path, subfolder, description, subfolder_path)

print("✅ All subfolders processed!")
